In [10]:
import spacy
import csv
import re
from subprocess import Popen
from subprocess import PIPE

In [11]:
def identifyEntity(sentence):
    sentence = sentence.replace(",", "")
    sentence = sentence.replace(".", "")
    lstEntities = []
    for word in sentence.split():
        if word in dictEntities.values():
            if word not in lstEntities:
                lstEntities.append(word)
    return lstEntities

In [12]:
nlp = spacy.load('es_core_news_lg')

In [13]:
dictEntities = {}
with open('../listRelacion.csv') as file: 
    reader = csv.reader(file)
    for row in reader:
        if 'Entidad' == row[0]: continue
        key = row[0]
        dictEntities[key] = row[1:][0]

In [14]:
with open('../textoLeySust1.txt') as file:
    data = file.read()

In [15]:
doc = nlp(data)

In [16]:
lstSentiments = [['Entidad', 'Enunciado', 'Puntuación Positiva', 'Puntuación Negativa',
                  'Palabras que dan la carga']]
for sentence in list(doc.sents):
    identifySentiment = 0
    for entity in identifyEntity(str(sentence)):
        if not identifySentiment:
            identifySentiment = 1
            sentStrength = Popen(["java", "-jar", "./java/SentStrength/SentiStrengthCom.jar", "sentidata", 
                                  "./java/SentStrength_Data/Español/", "text", str(sentence), "sentenceCombineTot", 
                                  "paragraphCombineTot", "explain", "trinary"], stdout = PIPE)
            result = sentStrength.communicate()[0].decode('utf-8')
            polarity = result.split()[0:2]
            wordsPolarity = re.findall(r'(\w+\[\-?[0-9]+\])', result)
        
        lstSentiments.append([entity, sentence, polarity[0], polarity[1], wordsPolarity])

In [17]:
with open('sentencesPolarity.csv', 'w', encoding = 'utf_8_sig') as file:
    writer = csv.writer(file)
    writer.writerows(lstSentiments)